# Tarea APIs

### Instrucciones:
    - Extraer l os resúmenes de artículos de wikipedia que contenga la palbra 'astro' (limitar a máximo 20).
    - Crear una variable con 0 o 1 que indique si el resumen contiene alguna de las palabras 'fisica' o 'ciencia'.
    - Crear dataframe con columnas: pageid, titulo, resumen, contiene_palabra.
    
Con esto podemos tener una idea (burda) de si el contenido es científico. Pueden elegir cualquier otra palabra para seleccionar los artículos y para la búsqueda dentro del resumen (o todo el texto del artículo).

(utilizar la API rest para obtener la lista de páginas).

In [1]:
#Comenzamos por el primer punto para obtener los resúmenes(...)

#Importamos librerias
import pandas as pd
import requests

In [2]:
#Buscamos la palabra astro
palabra = 'astro'
#variable con el url
endpoint = 'https://es.wikipedia.org/w/rest.php/v1/search/title'
#creamos nuestro duccionario
dic = {'q': palabra, 'limit': 20}
#Realizamos la petición
request = requests.get(endpoint, params=dic)
#Verificamos el status
request.status_code

200

In [6]:
#Verificamos en un dataframe
df = pd.json_normalize(request.json()['pages'])
#df

In [9]:
# Obtener los títulos y las páginas en listas
titles_list = df['title'].tolist()
pages_list = df['id'].tolist()

# Crear la lista de extractos
extracts_list = []

# Realizar las solicitudes a la API de Wikipedia
for i in range(len(pages_list)):
    page = pages_list[i]
    title = titles_list[i]
    url = f'https://es.wikipedia.org/w/api.php?action=query&format=json&pageids={page}&prop=extracts&explaintext=True&exintro=True'
    request_wiki = requests.get(url)
    data = request_wiki.json()
    extract = data['query']['pages'][str(page)]['extract']
    extracts_list.append(extract)

# Crear el diccionario y convertirlo en DataFrame
extracts_dict = {'title': titles_list, 'page_id': pages_list, 'extract': extracts_list}
extracts_df = pd.DataFrame(extracts_dict)

In [10]:
extracts_df

,title,page_id,extract
0,Objeto astronómico,59546,Un objeto astronómico es una entidad física si...
1,Astronomía,261,La astronomía (del griego άστρον [ástron] 'est...
2,Astrología,266,"La astrología, en su acepción más amplia, es u..."
3,Astronomía maya,9149789,La astronomía maya forma parte de una tradició...
4,Astrología china,257842,La astrología china es la astrología del calen...
5,Astro (grupo musical),7411986,"Astro (en hangul, 아스트로) (estilizado como ASTRO..."
6,Astronáutica,15216,La astronáutica es la teoría y práctica de la ...
7,Astrobiología,6007265,La astrobiología es la rama de la biología y l...
8,Astronauta,3323,Un astronauta[1]​ o cosmonauta[2]​ es una pers...
9,Astronomía en Chile,1779766,La astronomía en Chile ha sido desarrollada en...


In [11]:
# Verificar si las palabras "ciencia" o "física" están en la columna "extract"
Fisica_Ciencia_Binary = (extracts_df['extract'].str.contains('ciencia', case=False) | extracts_df['extract'].str.contains('física', case=False)).astype(int).tolist()
Fisica_Ciencia_Binary

[1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0]

In [12]:
# Convertir la lista de resultados a una serie de pandas
Fisica_Ciencia_Binary = pd.Series(Fisica_Ciencia_Binary)

# Unir la serie de resultados al dataframe original y renombrar a 'contiene_palabra'
extracts_df = pd.concat([extracts_df, Fisica_Ciencia_Binary.rename('contiene_palabra')], axis=1)
extracts_df


,title,page_id,extract,contiene_palabra
0,Objeto astronómico,59546,Un objeto astronómico es una entidad física si...,1
1,Astronomía,261,La astronomía (del griego άστρον [ástron] 'est...,1
2,Astrología,266,"La astrología, en su acepción más amplia, es u...",1
3,Astronomía maya,9149789,La astronomía maya forma parte de una tradició...,0
4,Astrología china,257842,La astrología china es la astrología del calen...,0
5,Astro (grupo musical),7411986,"Astro (en hangul, 아스트로) (estilizado como ASTRO...",0
6,Astronáutica,15216,La astronáutica es la teoría y práctica de la ...,1
7,Astrobiología,6007265,La astrobiología es la rama de la biología y l...,1
8,Astronauta,3323,Un astronauta[1]​ o cosmonauta[2]​ es una pers...,0
9,Astronomía en Chile,1779766,La astronomía en Chile ha sido desarrollada en...,1
